In [1]:
import pandas as pd

from google.cloud import storage

In [2]:
data = pd.read_csv('https://controllerdata.lacity.org/api/views/g9h8-fvhu/rows.csv', low_memory=False)

In [3]:
data.head()

,RECORD_NBR,PAY_YEAR,DEPARTMENT_NO,DEPARTMENT_TITLE,JOB_CLASS_PGRADE,JOB_TITLE,EMPLOYMENT_TYPE,JOB_STATUS,MOU,MOU_TITLE,REGULAR_PAY,OVERTIME_PAY,ALL_OTHER_PAY,TOTAL_PAY,CITY_RETIREMENT_CONTRIBUTIONS,BENEFIT_PAY,GENDER,ETHNICITY
0,303030303632,2017,98,WATER AND POWER,3156-5,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,55725.24,4785.05,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC
1,3030303036,2017,98,WATER AND POWER,9105-5,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,139174.88,16340.50,6170.49,161685.87,9186.0,23508.9,FEMALE,ASIAN AMERICAN
2,303030313232,2017,98,WATER AND POWER,9602-4,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,245879.12,0.00,12504.30,258383.42,16228.0,23508.9,MALE,BLACK
3,303030313632,2017,98,WATER AND POWER,5885-5,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,101494.34,7824.99,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN
4,303030323632,2017,98,WATER AND POWER,3841-5,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,101345.12,22284.37,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC
